In [1]:
using Plots
using Statistics

# Functions 

In [2]:
function F(t, tau, tau_1, A)

    if mod(t, tau) >= 0 && mod(t, tau) < tau_1

        return A

    else

        return -A
        
    end
    
end

function F_2(t, A, omega)
   
    return A * sin(omega * t)
    
end

function q(x, t, L, a, Q, lambda, tau, tau_1, A)

    if mod(x, L) >= 0 && mod(x, L) < a

        return -lambda - Q / a + F(t, tau, tau_1, A)

    else

        return -lambda + Q / (1 - a) + F(t, tau, tau_1, A)
        
    end
    
end

function q_2(x, t, L, a, Q, lambda, A, omega)

    if mod(x, L) >= 0 && mod(x, L) < a

        return -lambda - Q / a + F_2(t, A, omega)

    else

        return -lambda + Q / (1 - a) + F_2(t, A, omega)
        
    end
    
end

function g(x, T)

    return sqrt(2 * T)
    
end

function Milshtein(T, lambda, L, a, Q, A, tau, tau_1, h, tf, times)
    
    N = Int(floor(tf/h))

    sqrt_h = sqrt(h)
    
    x_arr = zeros(N)
    E_arr = zeros(N)
    
    for k in 1 : times
        
        x = 0
        t = 0
    
        @inbounds for i in 1 : N

            t += h
            
            rand_g = randn()

            x_i = h * q(x, t, L, a, Q, lambda, tau, tau_1, A) + g(x, T) * sqrt_h * rand_g
            
            x += x_i

            x_arr[i] += x
            E_arr[i] += F(t, tau, tau_1, A) * x_i / h

        end
        
    end
    
    return x_arr ./ times, E_arr ./ times
    
end

function Milshtein_2(T, lambda, L, a, Q, A, omega, h, tf, times)
    
    N = Int(floor(tf/h))

    sqrt_h = sqrt(h)
    
    x_arr = zeros(N)
    E_arr = zeros(N)
    
    for k in 1 : times
        
        x = 0
        t = 0
    
        @inbounds for i in 1 : N

            t += h
            
            rand_g = randn()

            x_i = h * q_2(x, t, L, a, Q, lambda, A, omega) + g(x, T) * sqrt_h * rand_g
            
            x += x_i

            x_arr[i] += x
            E_arr[i] += F_2(t, A, omega) * x_i / h

        end
        
    end
    
    return x_arr ./ times, E_arr ./ times
    
end

function J_T(Ts, lambda, L, a, Q, A, tau, tau_1, h, tf, times)
   
    Js = zeros(length(Ts))
    etas = zeros(length(Ts))
    
    errors_J = zeros(length(Ts))
    errors_etas = zeros(length(Ts))
    
    f = open("Results_A_$A.txt", "w")
    
    println(f, "#T\tJ\terr_J\tη\terr_η")
    
    i = 0
    
    @inbounds for T in Ts

        i += 1

        xs, Es = Milshtein(T, lambda, L, a, Q, A, tau, tau_1, h, tf, times);

        vels = ((circshift(xs, 1) - xs) ./ h)[2 : end];

        Js[i] = abs(mean(vels))
        errors_J[i] = sqrt(var(vels) / length(vels))
        
        etas[i] = lambda * Js[i] / abs(mean(Es))
        errors_etas[i] = lambda * errors_J[i] / abs(mean(Es))
        
        println(f, T, "\t", Js[i], "\t", errors_J[i], "\t", etas[i], "\t", errors_etas[i])
    
    end
    
    close(f)
    
    return Js, etas
    
end

function J_T_dif(Ts, lambda, L, a, Q, A, tau, tau_1, h, tf, times, filename)
   
    J_A = zeros(length(Ts))
    J_A_neg = zeros(length(Ts))
    J_ratio = zeros(length(Ts))
    
    f = open(filename, "w")
    
    println(f, "#T\tJ_A\tJ_A_neg\tJ_ratio")
    
    i = 0
    
    @inbounds for T in Ts

        i += 1

        xs, Es = Milshtein(T, lambda, L, a, Q, A, tau, tau_1, h, tf, times)

        vels = ((circshift(xs, 1) - xs) ./ h)[2 : end]
        
        for k in 1 : length(vels)
            
            if mod(k*h, tau) >= 0 && mod(k*h, tau) < tau_1

                J_A[i] += -vels[k]

            else

                J_A_neg[i] += -vels[k]

            end
            
        end
        
        J_A[i] = J_A[i] / (length(vels) * tau_1 / tau)
            
        J_A_neg[i] = J_A_neg[i] / (length(vels) * tau_1 / tau)
        
        J_ratio[i] = abs(J_A_neg[i] / J_A[i])
        
        println(f, T, "\t", J_A[i], "\t", J_A_neg[i], "\t", J_ratio[i])
    
    end
    
    close(f)
    
    return J_A, J_A_neg, J_ratio
    
end

function J_A(As, T, lambda, L, a, Q, tau, tau_1, h, tf, times)
   
    Js = zeros(length(As))
    etas = zeros(length(As))
    
    errors_J = zeros(length(As))
    errors_etas = zeros(length(As))
    
    f = open("Results_T_$T.txt", "w")
    
    println(f, "#A\tJ\terr_J\tη\terr_η")
    
    i = 0
    
    @inbounds for A in As

        i += 1

        xs, Es = Milshtein(T, lambda, L, a, Q, A, tau, tau_1, h, tf, times);

        vels = ((circshift(xs, 1) - xs) ./ h)[2 : end];

        Js[i] = abs(mean(vels))
        errors_J[i] = sqrt(var(vels) / length(vels))
        
        etas[i] = lambda * Js[i] / abs(mean(Es))
        errors_etas[i] = lambda * errors_J[i] / abs(mean(Es))
        
        println(f, A, "\t", Js[i], "\t", errors_J[i], "\t", etas[i], "\t", errors_etas[i])
    
    end
    
    close(f)
    
    return Js, etas
    
end

function J_taus(taus_1, T, lambda, L, a, Q, A, tau, h, tf, times)
   
    Js = zeros(length(taus_1))
    etas = zeros(length(taus_1))
    
    errors_J = zeros(length(taus_1))
    errors_etas = zeros(length(taus_1))
    
    f = open("Results_A_$A.txt", "w")
    
    println(f, "#tau_1/tau_2\tJ\terr_J\tη\terr_η")
    
    i = 0
    
    @inbounds for tau_1 in taus_1
        
        ratio = tau_1 / (tau - tau_1)

        i += 1

        xs, Es = Milshtein(T, lambda, L, a, Q, A, tau, tau_1, h, tf, times);

        vels = ((circshift(xs, 1) - xs) ./ h)[2 : end];

        Js[i] = -mean(vels)
        errors_J[i] = sqrt(var(vels) / length(vels))
        
        etas[i] = lambda * Js[i] / abs(mean(Es))
        errors_etas[i] = lambda * errors_J[i] / abs(mean(Es))
        
        println(f, ratio, "\t", Js[i], "\t", errors_J[i], "\t", etas[i], "\t", errors_etas[i])
    
    end
    
    close(f)
    
    return Js, etas
    
end

function J_T_2(Ts, lambda, L, a, Q, A, omega, h, tf, times)
   
    Js = zeros(length(Ts))
    etas = zeros(length(Ts))
    
    errors_J = zeros(length(Ts))
    errors_etas = zeros(length(Ts))
    
    f = open("Results_ω_A_$A.txt", "w")
    
    println(f, "#T\tJ\terr_J\tη\terr_η")
    
    i = 0
    
    @inbounds for T in Ts

        i += 1

        xs, Es = Milshtein_2(T, lambda, L, a, Q, A, omega, h, tf, times);

        vels = ((circshift(xs, 1) - xs) ./ h)[2 : end];

        Js[i] = abs(mean(vels))
        errors_J[i] = sqrt(var(vels) / length(vels))
        
        etas[i] = lambda * Js[i] / abs(mean(Es))
        errors_etas[i] = lambda * errors_J[i] / abs(mean(Es))
        
        println(f, T, "\t", Js[i], "\t", errors_J[i], "\t", etas[i], "\t", errors_etas[i])
    
    end
    
    close(f)
    
    return Js, etas
    
end

J_T_2 (generic function with 1 method)

In [3]:
#Fixed parameters
lambda = 0.01
L = 1
a = 0.8
Q = 1

tau = 6
tau_1 = tau / 2

h = 10^-3
tf = 50 * tau
times = 10^3

omega = pi / 3

#Parameters

T1 = collect(0.01 : 0.01 : 0.1)
T2 = collect(0.15 : 0.05 : 0.6)

Ts = vcat(T1, T2)

A = 3

filename = "results_A_3_tau_6.txt"

Js, etas = @time J_T_dif(Ts, lambda, L, a, Q, A, tau, tau_1, h, tf, times, filename);    

267.057278 seconds (1.37 M allocations: 413.427 MiB, 0.04% gc time)
